In [2]:
import pandas as pd 
import ta  
import numpy as np  
from tqdm import tqdm  
import pandas_ta as pda  
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib import cm 
from mpl_toolkits.mplot3d import axes3d

In [5]:
coin_dash=pd.read_csv("/Users/midou/Desktop/dashbord11_coin.csv")  
coin_dash=coin_dash.set_index(coin_dash["timestamp"])  
coin_dash_train=coin_dash[:500] #'2020-8-10'
coin_dash_test=coin_dash[500:]
data=[coin_dash_train ,coin_dash_test]  
paire_name = ["XRPUSDT","BTCUSDT","DASHUSDT","ZRXUSDT","ETHUSDT","XMRUSDT","LTCUSDT"] 
ded=[0,1]


In [6]:
def simule_super(df , name):  
    simule =pd.DataFrame()
    rowslist =[] 
    myrange=np.arange(1.0,20.0,3.5)
    for i in range(10,150,10): 
        for j in range(10 ,200,20) : 
            for z in myrange:
                coin=0 
                m=1000  
                ST_length = j
                ST_multiplier = z
                superTrend = pda.supertrend(df['high '+name], df['low '+name], df['close '+name], 
                                        length=ST_length, multiplier=ST_multiplier)
                df['SUPER_TREND'] = superTrend['SUPERT_'+str(ST_length)+"_"+str(ST_multiplier)]
                df['SUPER_TREND_DIRECTION1'] = superTrend['SUPERTd_'+str(ST_length)+"_"+str(ST_multiplier)]
                
                ST_length = j*2
                ST_multiplier = z*2.0
                superTrend = pda.supertrend(df['high '+name], df['low '+name], df['close '+name],
                                        length=ST_length, multiplier=ST_multiplier)
                df['SUPER_TREND'] = superTrend['SUPERT_'+str(ST_length)+"_"+str(ST_multiplier)]
                df['SUPER_TREND_DIRECTION2'] = superTrend['SUPERTd_'+str(ST_length)+"_"+str(ST_multiplier)]

                ST_length = j*2
                ST_multiplier = z*2.0
                superTrend = pda.supertrend(df['high '+name], df['low '+name], df['close '+name], 
                                        length=ST_length, multiplier=ST_multiplier)
                df['SUPER_TREND'] = superTrend['SUPERT_'+str(ST_length)+"_"+str(ST_multiplier)]
                df['SUPER_TREND_DIRECTION3'] = superTrend['SUPERTd_'+str(ST_length)+"_"+str(ST_multiplier)]
                df['EMA50']=ta.trend.sma_indicator(df['close '+name], i) 
                df['STOCH_RSI']=ta.momentum.stochrsi(df["close "+name] , window=14 ,smooth1=3 ,smooth2=3) 
                rotter=[]
               
                for index, row in df.iterrows() : 
                    if (row['SUPER_TREND_DIRECTION1']+row['SUPER_TREND_DIRECTION2']+row['SUPER_TREND_DIRECTION3'] >= 1 
                       and row['STOCH_RSI'] < 0.8 and row['close '+name]>row['EMA50'] and m > 0): 
                        coin=m/df['close '+name][index]  
                        coin=coin-coin*0.007 
                        bot_portefeuille = m +coin*df['close '+name][index]
                        chik={'i '+name:i , 'j '+name:j,'z '+name: z ,'value '+name:bot_portefeuille}
                        rotter.append(chik)  
                
                    if (row['SUPER_TREND_DIRECTION1']+row['SUPER_TREND_DIRECTION2']+row['SUPER_TREND_DIRECTION3'] < 1 
                        and row['STOCH_RSI'] > 0.2 and coin > 0):  
                        m=coin*df['close '+name][index] 
                        m=m-m*0.007 
                        coin=0 
                        bot_portefeuille = m +coin*df['close '+name][index]  
                        chik={'i '+name :i , 'j '+name:j ,'z '+name: z,' value '+name :bot_portefeuille}  
                        rotter.append(chik)  
                 
                dt=pd.DataFrame(rotter)  
                clock=dt.iloc[-1:]  
                simule=simule.append(clock)  
    return simule

In [7]:
def most_per(df , name ,n) :
    df=df.sort_values(by='value '+name, ascending=False) 
    df=df[:n] 
    return df 

In [20]:
for name in tqdm(paire_name) :  
    for d in ded :  
        if d==0 :  
            train=simule_super (data[d] ,name)  
            result_train=most_per(train , name ,30) 
            #print('train result',result)
            
        if d==1 :   
            test=simule_super (data[d] ,name)   
            result_test=most_per(train , name, 50) 
            #print('test result', result ,75)  
    count=0
    for index, row in result_train.iterrows(): 
        for index ,row2 in result_test.iterrows(): 
            if (row['i '+name]==row2['i '+name] and row['j '+name]==row2['j '+name] 
                and row['z '+name]==row2['z '+name] ) : 
                        while count<=0 :
                              print(row2)  #print row which do performance in train and in test 
                              count+=1
                
            else : 
                pass

  0%|          | 0/7 [00:00<?, ?it/s]/var/folders/4k/2v2nn1sn1k1f7dtmr6ny96vh0000gn/T/ipykernel_7462/1402080350.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SUPER_TREND'] = superTrend['SUPERT_'+str(ST_length)+"_"+str(ST_multiplier)]
/var/folders/4k/2v2nn1sn1k1f7dtmr6ny96vh0000gn/T/ipykernel_7462/1402080350.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SUPER_TREND_DIRECTION1'] = superTrend['SUPERTd_'+str(ST_length)+"_"+str(ST_multiplier)]
/var/folders/4k/2v2nn1sn1k1f7dtmr6ny96vh0000gn/

i XRPUSDT          140.0
j XRPUSDT          190.0
z XRPUSDT           18.5
value XRPUSDT     1993.0
 value XRPUSDT       NaN
Name: 39, dtype: float64


 29%|██▊       | 2/7 [07:21<18:24, 220.92s/it]

i BTCUSDT           80.000000
j BTCUSDT           10.000000
z BTCUSDT            1.000000
value BTCUSDT     2314.663888
 value BTCUSDT            NaN
Name: 108, dtype: float64


 43%|████▎     | 3/7 [11:01<14:41, 220.37s/it]

...

i ETHUSDT          140.0
j ETHUSDT          190.0
z ETHUSDT           18.5
value ETHUSDT     1993.0
 value ETHUSDT       NaN
Name: 120, dtype: float64


 86%|████████▌ | 6/7 [22:02<03:40, 220.24s/it]

i XMRUSDT          140.0
j XMRUSDT          190.0
z XMRUSDT           18.5
value XMRUSDT     1993.0
 value XMRUSDT       NaN
Name: 116, dtype: float64


100%|██████████| 7/7 [25:39<00:00, 219.98s/it]

i LTCUSDT          140.0
j LTCUSDT          190.0
z LTCUSDT           18.5
value LTCUSDT     1993.0
 value LTCUSDT       NaN
Name: 46, dtype: float64


In [39]:
#i_superT=[140,80,60 ,80,140,140]
#j_superT=[190,10,190,10,190 ,190]
#z_superT=[18.5,1.0,1.0,18.5,18.5,18.5] 


In [30]:
def simule_AO (df ,name) :  
    simule=pd.DataFrame() 
    simule_volume_strg=pd.DataFrame() 
    for i in range(1,100,3) : 
        for j in range(10,50,5):
            rowslist =[]  
            m=1000
            coin=0
       
            df['STOCH_RSI'] = ta.momentum.stochrsi(close=df['close '+name], window=14) 
            df['WillR'] = ta.momentum.williams_r(high=df['high '+name], low=df['low '+name], 
                                                    close=df['close '+name],lbp=14) 
            df['AO']= ta.momentum.awesome_oscillator(df['high '+name],df['low '+name],window1=i
                                                        ,window2=j)  
            rotter=[]   
    
            for index, row in df.iterrows() : 
                if row['AO'] >= 0 and row['WillR'] < -85 and m>0:  
                    coin=m/row['close '+name]  
                    coin=coin-coin*0.007  
                    m=0
                    bot_portefeuille = m +coin*row['close '+name] 
                    chik={'i '+name:i , 'j '+name:j ,'value '+name:bot_portefeuille}
                    rotter.append(chik)  
     
                if row['AO'] < 0 and row['STOCH_RSI'] > 0.2 and coin>0.001 :  
                    m=coin*row['close '+name] 
                    m=m-m*0.007 
                    coin=0   
                    bot_portefeuille = m +coin*row['close '+name]  
                    chik={'i '+name:i , 'j '+name:j,'value '+name:bot_portefeuille}
                    rotter.append(chik)  
            dt=pd.DataFrame(rotter)  
            clock=dt.iloc[-1:]  
            simule =simule.append(clock) 
    return simule

In [38]:
for name in tqdm(paire_name) :    
      
    train=simule_AO (coin_dash_train ,name)  
    result_train=most_per(train , name ,30) 
    #print('train result',result)
              
    test=simule_AO(coin_dash_test ,name)   
    result_test=most_per(train , name, 50) 
    #print('test result', result ,75)  
    count=0
    for index, row in result_train.iterrows(): 
        for index ,row2 in result_test.iterrows(): 
            if row['i '+name]==row2['i '+name] and row['j '+name]==row2['j '+name] : 
                    while count<=0 :
                        print(row2)  #print row which do performance in train and in test 
                        count+=1
            else : 
                pass

  0%|          | 0/7 [00:00<?, ?it/s]/var/folders/4k/2v2nn1sn1k1f7dtmr6ny96vh0000gn/T/ipykernel_7462/57776629.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STOCH_RSI'] = ta.momentum.stochrsi(close=df['close '+name], window=14)
/var/folders/4k/2v2nn1sn1k1f7dtmr6ny96vh0000gn/T/ipykernel_7462/57776629.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['WillR'] = ta.momentum.williams_r(high=df['high '+name], low=df['low '+name],
/var/folders/4k/2v2nn1sn1k1f7dtmr6ny96vh0000gn/T/ipykernel_7462/577

i XRPUSDT          61.000000
j XRPUSDT          25.000000
value XRPUSDT    1630.728129
Name: 7, dtype: float64


 29%|██▊       | 2/7 [00:30<01:16, 15.34s/it]

i BTCUSDT          16.000000
j BTCUSDT          25.000000
value BTCUSDT    2023.865283
Name: 17, dtype: float64


 43%|████▎     | 3/7 [00:45<00:59, 15.00s/it]

...

i ETHUSDT          49.000000
j ETHUSDT          15.000000
value ETHUSDT    2396.258542
Name: 7, dtype: float64


 86%|████████▌ | 6/7 [01:30<00:15, 15.01s/it]

i XMRUSDT          46.000000
j XMRUSDT          30.000000
value XMRUSDT    2100.922766
Name: 7, dtype: float64


100%|██████████| 7/7 [01:46<00:00, 15.18s/it]

i LTCUSDT           7.00000
j LTCUSDT          20.00000
value LTCUSDT    2422.09854
Name: 9, dtype: float64


In [ ]:
## WE TAKE RESULT FOR INDICATORS IMPLIMENTATION 